# Model 1: meta-llama/Meta-Llama-3-8B-Instruct

In [1]:
!pip install -U accelerate transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 66.8 MB/s eta 0:00:00


In [2]:
!pip install -i https://pypi.org/simple/ bitsandbytes --upgrade --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.1 MB/s eta 0:00:00


In [3]:
!pip install accelerate --q

In [1]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [2]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, AutoConfig, BitsAndBytesConfig

In [3]:
q_config = BitsAndBytesConfig (
      load_in_4bit = True,
      bnb_4bit_quant_type="nf4",
  )

In [4]:
def create_model_tokenizer(name, q_config):
  tokenizer = AutoTokenizer.from_pretrained(name)
  model = AutoModelForCausalLM.from_pretrained (
      name,
      quantization_config = q_config,
      device_map="cuda",
      torch_dtype="auto",
      trust_remote_code = True
  )
  return model, tokenizer

In [5]:
# Load model directly
model1, tokenizer1 = create_model_tokenizer("NousResearch/Meta-Llama-3-8B-Instruct", q_config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [6]:
generation_pipeline = pipeline(
    "text-generation",
    model=model1,
    tokenizer=tokenizer1,
    framework="pt",
)

In [9]:
def are_questions_similar(question1, question2):
    input_text = f"Question 1: {question1}\nQuestion 2: {question2}\n"
    output = generation_pipeline(input_text, max_length=10, num_return_sequences=1, max_new_tokens=20)
    generated_text = output[0]['generated_text'].strip()
    return generated_text

In [10]:
# Example usage
question1 = "What is the capital of France?"
question2 = "Which city is the capital of France?"
generated_text = are_questions_similar(question1, question2)
print(f"Generated text: {generated_text}")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Both `max_new_tokens` (=20) and `max_length`(=10) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generated text: Question 1: What is the capital of France?
Question 2: Which city is the capital of France?
?

A) Paris
B) Berlin
C) London
D) Rome

Answer: A
